# Use TensorFlow 2.x.

In [0]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)

# Install Tensorflow-Addons.
*   Instance Normalization

In [0]:
!pip install tensorflow-addons

# Install Tensorflow-datasets.

In [0]:
!pip install tensorflow-datasets

# List available TensorFlow datasets.

In [0]:
import tensorflow_datasets as tfds

tfds.list_builders()